## Features extraction

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import torchaudio
import torch
from transformers import (
    Wav2Vec2FeatureExtractor, Wav2Vec2Model,
    AutoTokenizer, AutoModelForSequenceClassification,
    AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline,
    RobertaTokenizer, RobertaModel
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ROOT_AUDIO_DIR = "/content/drive/MyDrive/speech_wav/audio"
TRANSCRIPTS_PATH = "/content/features.csv"
LABELS_PATH = "/content/features.csv"
OUTPUT_FEATURE_FILE = "/content/wav2vec_features.npy"
OUTPUT_NAMES_FILE = "/content/file_names.csv"
MAX_LENGTH = 300
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Mounted at /content/drive


### Audio: Wav2Vec2 model

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2FeatureExtractor # Import Wav2Vec2FeatureExtractor
import torch

# Load Wav2Vec2
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")
wav2vec_model = Wav2Vec2Model.from_pretrained("ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition").to(DEVICE)
wav2vec_model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:311: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (1-4): 4 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2LayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,))
        (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=1024, bias=True)
    (dropout)

In [ ]:
# Extract Audio Features
def extract_wav2vec_features(audio_path, max_len=300):
    waveform, sr = torchaudio.load(audio_path)
    if sr != 16000:
        waveform = torchaudio.transforms.Resample(sr, 16000)(waveform)
    inputs = feature_extractor(waveform.squeeze(), sampling_rate=16000, return_tensors="pt")
    with torch.no_grad():
        outputs = wav2vec_model(**inputs.to(DEVICE))
    features = outputs.last_hidden_state.squeeze(0).cpu().numpy()
    if features.shape[0] < max_len:
        features = np.pad(features, ((0, max_len - features.shape[0]), (0, 0)), mode='constant')
    else:
        features = features[:max_len]
    return features

### text: Roberta

In [ ]:
!pip install transformers
from transformers import RobertaTokenizer, RobertaModel

# Load roberta
bert_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
bert_model = RobertaModel.from_pretrained("roberta-base").to(DEVICE)
bert_model.eval()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [ ]:
# Extract Text Features
def extract_text_features(text):
    inputs = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = bert_model(**inputs.to(DEVICE))
    return outputs.last_hidden_state[:, 0, :].squeeze(0).cpu().numpy()  # [CLS] token output

## audio + text

In [ ]:
# Load Transcripts
transcripts_df = pd.read_excel('/content/drive/MyDrive/speech_wav/text_sentences.xlsx')

# Detect correct column for audio matching
print("Transcript columns:", transcripts_df.columns.tolist())
audio_col = None
for col in transcripts_df.columns:
    if col.lower() in ["audio", "file", "clip", "filename", "clipname"]:
        audio_col = col
        break
if audio_col is None:
    raise ValueError("No suitable column found for matching audio file names in transcript file.")

Transcript columns: ['clipName', 'transcript', 'group', 'word', 'tone', 'indicator', 'type', 'sentence', 'word_count']


In [ ]:
# Extract Features
features_audio, features_text, file_names = [], [], []
for folder in os.listdir(ROOT_AUDIO_DIR):
    folder_path = os.path.join(ROOT_AUDIO_DIR, folder)
    if not os.path.isdir(folder_path):
        continue
    for filename in tqdm(os.listdir(folder_path), desc=f"Processing {folder}"):
        if not filename.endswith(".wav"):
            continue
        clip_name = filename[:-4].strip()
        path = os.path.join(folder_path, filename)
        try:
            audio_feat = extract_wav2vec_features(path)
            text_row = transcripts_df[transcripts_df[audio_col] == clip_name]
            if len(text_row) == 0:
                print(f"Transcript not found for: {clip_name}")
                continue
            text_feat = extract_text_features(text_row.iloc[0]["transcript"])
            features_audio.append(audio_feat)
            features_text.append(text_feat)
            file_names.append(clip_name)
        except Exception as e:
            print(f" Failed: {clip_name} — {e}")

In [ ]:
features_audio = np.array(features_audio)
features_text = np.array(features_text)
pd.DataFrame({"audio": file_names}).to_csv(OUTPUT_NAMES_FILE, index=False)
np.save("/content/features_audio.npy", features_audio)
np.save("/content/features_text.npy", features_text)

print(f"Successfully extracted features for {len(file_names)} samples.")

In [ ]:
features_audio = np.load("/content/features_audio.npy")
# Reshape to 2D: (samples, features)
features_audio_2d = features_audio.reshape(features_audio.shape[0], -1)
feature_audio_df = pd.DataFrame(features_audio_2d)
print(feature_audio_df.head()) # Changed df_audio to feature_audio_df

In [ ]:
features_text = np.load("/content/features_text.npy")
# Reshape to 2D: (samples, features)
features_text_2d = features_text.reshape(features_text.shape[0], -1)
feature_text_df = pd.DataFrame(features_text_2d)
print(feature_text_df.head())

In [ ]:
features_df = pd.concat([feature_text_df, feature_audio_df], axis=1)
print(features_df.head())